# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project2
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

## 1st Query:
- We will name the table `song_history`
- The table contains a composite primary key which includes sessionID as a partition, and itemInSession as a clustering key as we will need to filter on these attributes.
- The columns of our table are: sessionId, itemInSession, artist, song_title and song_length.

In [8]:
query = "create table if not exists song_history_by_session_and_item"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, primary key((sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)          

#### Inserting data into `song_history_by_session_and_item`

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history_by_session_and_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Running a SELECT to check data

In [10]:
query = "select artist,song,length from song_history_by_session_and_item where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


## 2nd query:
- We create the table `song_history_by_user_and_session`
- Our primary key will consist of a composite partition key userId, sessionId. Because if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes, which will cause performance issue in case of voluminous data.
- Our clustering key will be itemInSession so that we are able to order our results by it.

In [11]:
query = "create table if not exists song_history_by_user_and_session "
query = query + "( userId int, sessionId int, itemInSession int, artist text, user text, song text, primary key((userId,sessionid),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Inserting values into `song_history_by_user_and_session`

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user = line[1]+" "+line[4]
        query = "INSERT INTO song_history_by_user_and_session (userId, sessionId, itemInSession, artist, user, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], user, line[9]))

### Running a SELECT to verify data

In [13]:
query = "select artist,song,user from song_history_by_user_and_session where userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz')
Row(artist='Three Drives', song='Greece 2000', user='Sylvie Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')


## 3rd Query:
- We create the table `user_by_song`
- Table has composite primary key which includes song as partition key and userId as clustering key because the query requires filtering by song, so song should be used in the where condition, that indicates that the data in table needs to be partitioned by song first. Then, in order to get unique data we included userId as the clustering key.
- Table coluns are: song, userId, user

In [14]:
query = "create table if not exists user_by_song "
query = query + "( song text, userId int, user text, primary key((song), userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Inserting values to `user_by_song`

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        user = line[1]+" "+line[4]
        query = "INSERT INTO user_by_song (song, userId, user)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), user))

#### Running a SELECT to verify data

In [16]:
query = "select song,user from user_by_song where song ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(song='All Hands Against His Own', user='Jacqueline Lynch')
Row(song='All Hands Against His Own', user='Tegan Levine')
Row(song='All Hands Against His Own', user='Sara Johnson')


### Drop the tables before closing out the sessions

In [17]:
query = "drop table song_history_by_session_and_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_history_by_user_and_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()